## Syntax-dependent trace

Head attribution

In [1]:
!nvidia-smi

Fri Jul  7 14:30:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|


|   0  NVIDIA H100 PCIe    Off  | 00000000:17:00.0 Off |                    0 |
| N/A   46C    P0    79W / 310W |  70149MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA H100 PCIe    Off  | 00000000:65:00.0 Off |                    0 |
| N/A   43C    P0    73W / 310W |  14665MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   2  NVIDIA H100 PCIe    Off  | 00000000:CA:00.0 Off |                    0 |
| N/A   45C    P0    79W / 310W |  73743MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   3  NVIDIA H100 PCIe    Off  | 000000

In [2]:
%load_ext autoreload
%autoreload 2
# %env CUDA_VISIBLE_DEVICES=2

# !echo $CUDA_VISIBLE_DEVICES

In [3]:
## discovery cluster setup
# CACHE_DIR = "/scratch/lucchetti.f/models/"
# os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

import os
import sys
sys.path.append('..')

import gc
import torch
from model_utils import *
import tqdm as notebook_tqdm
from trace_model import ModelLoader
# from data.make import int_data, random_data, str_data
torch.set_grad_enabled(False)
from tqdm import tqdm 
from model_utils import layername
#MODEL_NAME = "Salesforce/codegen-16B-mono"
MODEL_NAME = "bigcode/starcoder"
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import re
check_devs()
# clear_devs()
torch.cuda.is_available()

/home/franlucc/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 84978434048 used for device 0, reserved 0


True

In [4]:
lm = ModelLoader(MODEL_NAME)
check_devs()

Loading checkpoint shards:  29%|██▊       | 2/7 [00:07<00:19,  3.87s/it]

In [ ]:
from datasets import *

dataset = load_from_disk("../data/python_int_functions.parquet")["python"]
print("\n".join(dataset[0].split("\n")[-3:]))

            for input in tr.findall (".//%s" % tag ('input')) :
                name = input.get ('id').split ('.') [-1]



### Test trace_generate

In [10]:
prompts = ["\n".join(dataset[0].split("\n")[-3:])]


txt, ret_dict = lm.trace_generate(
    prompts,
    max_out_len=len(prompts[0]) + 10,
    pick_greedily= False,
    request_activations= [], #lm.model.config.n_layer
    request_logits= [layername(lm.model, i) for i in range(1, lm.model.config.n_layer)], #lm.model.config.n_layer
    top_k = 10
)

txt, ret_dict

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 prompts = ["\n".join(dataset[0].split("\n")[-3:])]                                          │
│    2                                                                                             │
│    3                                                                                             │
│ ❱  4 txt, ret_dict = lm.trace_generate(                                                          │
│    5 │   prompts,                                                                                │
│    6 │   max_out_len=len(prompts[0]) + 10,                                                       │
│    7 │   pick_greedily= False,                                                                   │
│                                                                                                  │
│ /home/franlucc/codetrace/notebooks/../trace_model.py:143 in trace_generate                       │
│                                                                                                  │
│   140 │   │   │   │   │   retain_output=True,                                                    │
│   141 │   │   │   │   │   stop=stop,                                                             │
│   142 │   │   │   │   ) as traces:                                                               │
│ ❱ 143 │   │   │   │   │   model_out = self.model(                                                │
│   144 │   │   │   │   │   │   input_ids=input_ids[:, cur_context],                               │
│   145 │   │   │   │   │   │   attention_mask=attention_mask[:, cur_context],                     │
│   146 │   │   │   │   │   │   past_key_values=past_key_values,                                   │
│                                                                                                  │
│ /home/franlucc/mechinterp/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in        │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /home/franlucc/mechinterp/lib/python3.10/site-packages/transformers/models/gpt_bigcode/modeling_ │
│ gpt_bigcode.py:808 in forward                                                                    │
│                                                                                                  │
│    805 │   │   """                                                                               │
│    806 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│    807 │   │                                                                                     │
│ ❱  808 │   │   transformer_outputs = self.transformer(                                           │
│    809 │   │   │   input_ids,                              

In [7]:
txt

['for input in tr.findall (".//%s" % tag (\'input\')) :\n                name = input.get (\'id\').split (\'.\') [0]\n                name, ext = name.rsplit (\'.\',1)\n                if name in self.inputs : self.inputs [name] [ext] = input\n\n            for input in tr.findall (".//%s" % tag (\'select\'))  :\n                name = input.get (\'id\')\n                name, ext   = name [1:].split (\'.\')\n                self.inputs']

## Experiment: 6th layer can predict int

Using python data

In [8]:
for ex in dataset[:5]:
    crop_to_int = "\n".join(ex.split("\n")[-2:])
    crop_to_int = re.findall(r"([\w\W]*?)\d", crop_to_int)[0]
    print(crop_to_int)
    txt, ret_dict = lm.trace_generate(
        [crop_to_int],
        max_out_len=len(crop_to_int) + 10,
        pick_greedily= False,
        request_activations= [], #lm.model.config.n_layer
        request_logits= [layername(lm.model, 6)], #lm.model.config.n_layer
        top_k = 10,
    )
    logits_layer_6 = [i for (i,_) in ret_dict["logits"][layername(lm.model, 6)]]
    print(logits_layer_6)
    if any(bool(re.search(r'\d', l)) for l in logits_layer_6):
        print("found")

    def tbl_iter (self, div) :
        tbl = div.find (".//%s" % tag ("table"))
        assert tbl.get ('class') == 'cbi-section-table'
        d = {}
        for tr in tbl :
            if 'cbi-section-table-row' not in tr.get ('class').split () :
                continue
            for input in tr.findall (".//%s" % tag ('input')) :
                name = input.get ('id').split ('.') [- True
['<fim_suffix>', ' )', '\n', ',', ' #', ' \\', '_', ' }', ' **', ' as']
    def table_iter (self) :
        root  = self.tree.getroot ()
        for div in root.findall (".//%s" % tag ("div")) :
            if div.get ('id') == 'maincontent' :
                break
        tbl = div.find (".//%s" % tag ("table"))
        if tbl is None :
            return
        for tr in tbl :
            if tr [ True
[' ', ' (', '<fim_suffix>', '  ', ' -', '.', '\n', '!', ' "', ' =']
class OLSR_Routes (Table_Iter) :
    url          = 'cgi-bin/luci/freifunk/olsr/routes'
    retries      =  True
['\n', ' ', '

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 for ex in dataset[:5]:                                                                      │
│    2 │   crop_to_int = re.findall(r"([\w\W]*?)\d", ex)[0]                                        │
│    3 │   print(crop_to_int, isinstance(crop_to_int, str))                                        │
│ ❱  4 │   txt, ret_dict = lm.trace_generate(                                                      │
│    5 │   │   [crop_to_int],                                                                      │
│    6 │   │   max_out_len=len(crop_to_int) + 10,                                                  │
│    7 │   │   pick_greedily= False,                                                               │
│                                                                                                  │
│ /home/franlucc/codetrace/notebooks/../trace_model.py:194 in trace_generate                       │
│                                                                                                  │
│   191 │   │   │   │                                                                              │
│   192 │   │   │   │   for i in range(input_ids.size(0)):                                         │
│   193 │   │   │   │   │   generated_tokens[i].append(                                            │
│ ❱ 194 │   │   │   │   │   │   [                                                                  │
│   195 │   │   │   │   │   │   │   {"token": self.tokenizer.decode(t), "id": t.item(), "p": sof   │
│   196 │   │   │   │   │   │   │   for t in tk[i]                                                 │
│   197 │   │   │   │   │   │   ]                                                                  │
│                                                                                                  │
│ /home/franlucc/codetrace/notebooks/../trace_model.py:195 in <listcomp>                           │
│                                                                                                  │
│   192 │   │   │   │   for i in range(input_ids.size(0)):                                         │
│   193 │   │   │   │   │   generated_tokens[i].append(                                            │
│   194 │   │   │   │   │   │   [                                                                  │
│ ❱ 195 │   │   │   │   │   │   │   {"token": self.tokenizer.decode(t), "id": t.item(), "p": sof   │
│   196 │   │   │   │   │   │   │   for t in tk[i]                                                 │
│   197 │   │   │   │   │   │   ]                                                                  │
│   198 │   │   │   │   │   )                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [9]:
check_devs()

78413496832 / 84978434048 used for device 0, reserved 79935045632
